# Poisson 1D

This notebook solves the linear problem:

$$ -u(x)'' = f(x) \text{ in } \Omega = [-1,1]$$ 

with

$$ f(x) = \frac{\pi^2}{4} \cos{\left(\frac{\pi x}{2}\right)} $$

and either homogeneous Dirichlet boundary conditions or mixed Dirichlet and Neumann using Hermite elements. The purpose is to find what the issues are while setting Dirichlet BCs and fix them. See [Manually rebuilding the mass matrix](#Manually-rebuilding-the-mass-matrix).

In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
import FIAT
%matplotlib inline

# Exact solution with homogeneous Dirichlet BCs:
u_exact = lambda x: np.cos(np.pi*x/2)

In [ ]:
def poisson1d(V):
    """ Solves -u'' = f on [-1,1] with f=... """
    deg = V.ufl_element().degree()
    left = V.mesh().coordinates().min()
    right = V.mesh().coordinates().max()

    def dirichlet(x):
        return near(x[0], left)# or near(x[0], right) #

    # Test: note that this changes the exact solution!
    class Neumann(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], right)

    NEUMANN_ID = 1
    exterior = FacetFunction('uint', V.mesh())
    neu_bdry = Neumann()
    neu_bdry.mark(exterior, NEUMANN_ID)
    
    # Need to project to work around lack of evaluate_dofs() for Hermite
    # (see interpolation.ipynb, interpolate_hermite())
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, dirichlet)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4", degree=deg)

    a = u.dx(0)*v.dx(0)*dx
    L = f*v*dx - v*ds(subdomain_data=exterior, subdomain_id=NEUMANN_ID)

    u = Function(V)
    solve(a == L, u, bc)
    return u

In [ ]:
#mesh = UnitIntervalMesh(2)
#MeshTransformation.translate(mesh, Point(1.0))
mesh = IntervalMesh(20, -1, 1)

V = FunctionSpace(mesh, "Lagrange", 3)
u_lag2 = poisson1d(V)

In [ ]:
xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u_lag2(x) for x in xx], label='$u_{l}$')
#pl.plot(xx, [u_exact(x) for x in xx], label='$u_{ex}$')
pl.ylim((0, 2))
pl.title("Solution with CG3 elements")
_ = pl.legend()

# Using Hermite elements

Naively applying Dirichlet boundary conditions fails because first derivatives are set to 0 when `DirichletBC` clears all the rows of dofs of nodes at the boundary, since this includes Hermite dofs associated to partial derivatives.

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d(V)

xx = np.linspace(-1, 1, 100)
pl.figure(figsize=(8,4))
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_lag2(x) for x in xx], label='$u_{l}$')
pl.ylim((0, 2))
_ = pl.legend()

## Manually rebuilding the stiffness matrix

The solution is to manually restore the relevant rows of the assembled matrix to their values before `DirichletBC.apply()` emptied them. The code has been factored out into [boundary.ipynb](boundary.ipynb).

Analogously, the way one applies Neumann boundary conditions has to be adapted in e.g. 4th order problems like the beam equation: in this case Neumann bcs are essential and are included in the function space by setting the relevant dofs to the desired values. See [Euler-Bernoulli.ipynb](Euler-Bernoulli.ipynb).

In [ ]:
import nbimporter
from boundary import *
from utils import fnor
from interpolation import interpolate_hermite, make_constant

def poisson1d_hermite(V):
    """
        V: FunctionSpace.
    """
    assert V.ufl_element().family() == 'Hermite', 'Please pay attention.'

    _left = V.mesh().coordinates().min()
    _right = V.mesh().coordinates().max()

    def left(x):
        return near(x[0], _left)
    def right(x):
        return near(x[0], _right)

    u0 = make_constant(0, V)
    dir_bc = DirichletBC(V, u0, left) #fnor(left, right))

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4",
                   degree=V.ufl_element().degree())

    #g = Expression("x[0]",
    #               degree=V.ufl_element().degree())

    class Right(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], _right)

    exterior = FacetFunction('uint', V.mesh(), value=0)
    neu_bdry = Right()
    neu_bdry.mark(exterior, 1)
    
    a = u.dx(0)*v.dx(0)*dx
    F = f*v*dx - v*ds(subdomain_data=exterior, subdomain_id=1)

    A = assemble(a)
    b = assemble(F)
    #Aok,bok = assemble_system(a, F, dir_bc)
    #print("A0:\n%s" % A.array().round(2))
    #print("Aok\n%s" % Aok.array().round(2))
    apply_dirichlet_hermite(A, b, dir_bc)
    #print("A1:\n%s" % A.array().round(2))
    
    u = Function(V)
    U = u.vector()
    solve(A, U, b)
    return u

In [ ]:
mesh = IntervalMesh(20, -1, 1)
V = FunctionSpace(mesh, 'Hermite', 3)
u_her = poisson1d_hermite(V)

In [ ]:
xx = mesh.coordinates()
pl.figure(figsize=(8,4))
pl.plot(xx, [u_her(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_lag2(x) for x in xx], label='$u_{lag2}$')
#pl.ylim((0, 1.1))
pl.ylim((0, 2))
#pl.xlim((0.9, 1))
_ = pl.legend()